In [1]:
# Cell to allow importation of script in parent folder
import sys
import os

current_dir = os.path.dirname(__file__) if '__file__' in globals() else os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_dir, "../")))

# EASE

...

In [2]:
from ease import EASE, FastEASE
from data_formatting import DataFormatting
import pandas as pd

%load_ext autoreload
%autoreload 2

In [3]:
data = DataFormatting('../data/train.csv')
X, maps = data.create_matrix_csr()
test_df = pd.read_csv("../data/test.csv")

In [4]:
ease = EASE(X=X, maps=maps)

In [5]:
ease.fit()

Computing the Gram-Matrix... [1/4]
Adding lambda to the diagonal of Gram-Matrix... [2/4]
Computing the inverse of Gram-Matrix... [3/4]
Compute the weight matrix... [4/4]

Training completed in 0:01:36.769145


In [6]:
ease.predict()

Starting to predict score of all (user,book) pairs...
Map score into a 1 to 5 score range...


  0%|          | 0/18905 [00:00<?, ?it/s]


Prediction completed in 0:01:23.155892


In [7]:
ease.retrieve_pred(df=test_df)

In [8]:
ease.save_pred()

In [9]:
ease.pred.max(), ease.pred.min(), ease.pred.mean()

(5.0, 1.0, 2.0646958507013418)

In [10]:
ease.score.max(), ease.score.min(), ease.score.mean()

(0.009893389399800648, -1.6726455434767776e-07, 6.049318282208454e-07)

In [11]:
ease.rating.max(), ease.rating.min(), ease.rating.mean()

(5.0, 1.0, 2.99999999999999)

In [12]:
look_results = pd.read_csv('../data/train.csv')
user_to_index = maps['user_to_index']
book_to_index = maps['item_to_index']
look_results['pred'] = look_results.apply(
    lambda row: ease.rating[user_to_index[row['user_id']], book_to_index[row['book_id']]], axis=1
)

In [13]:
look_results

,book_id,user_id,rating,pred
0,7260,20145,3.5,1.000509
1,243238,85182,4.0,4.102285
2,9135,45973,1.0,1.079180
3,18671,63554,3.0,1.000509
4,243293,81002,5.0,4.900707
...,...,...,...,...
100518,15374,69658,2.0,1.046337
100519,11063,69658,2.5,1.010693
100520,18444,29981,1.0,1.003055
100521,5917,38009,1.0,1.033352


In [14]:
import numpy as np
look_results.apply(lambda row: np.sqrt((row['rating'] - row['pred'])**2), axis=1).mean()

1.1700141757431115